In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jlvelaalonso","key":"72687d455a314ef5280bf2076c59622b"}'}

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!mv kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# === AUDITORÍA DE attack_cat EN DATASETS CSV ===
# Verifica qué clases existen realmente en cada CSV
# NO modifica datos (solo lectura)

import pandas as pd
from pathlib import Path

# ==================================================
# CONFIGURA AQUÍ LOS CSV QUE QUIERAS AUDITAR
# ==================================================
CSV_FILES = {
    "UNSW_REAL": "/gdrive/MyDrive/Datasets/UNSW_NB15_full_clean.csv",
    "UNSW_SYN":  "/gdrive/MyDrive/Datasets/synthetic_UNSW_NB15_ctgan.csv",

    "CICIDS_REAL": "/gdrive/MyDrive/Datasets/MachineLearningCVE_full_clean.csv",
    "CICIDS_SYN":  "/gdrive/MyDrive/Datasets/synthetic_MachineLearningCVE_ctgan.csv",

    "KITSUNE_REAL": "/gdrive/MyDrive/Datasets/KITSUNE_full_clean.csv",
    "KITSUNE_SYN":  "/gdrive/MyDrive/Datasets/synthetic_kitsune_ctgan.csv",
}

# ==================================================
def audit_csv(path: str, name: str):
    print("\n" + "="*70)
    print(f"[DATASET] {name}")
    print(f"[PATH]    {path}")

    try:
        df = pd.read_csv(path, low_memory=False)
    except Exception as e:
        print("❌ ERROR al leer CSV:", e)
        return

    print(f"[SHAPE]   {df.shape}")

    # columnas clave
    cols = df.columns.str.lower().tolist()
    has_attack = "attack_cat" in cols
    has_label  = "label" in cols

    print(f"[COLUMNAS] attack_cat={has_attack} | label={has_label}")

    if has_attack:
        ac = df["attack_cat"].astype(str).str.strip()
        vc = ac.value_counts(dropna=False)

        print("\n[attack_cat] Clases detectadas:")
        for k, v in vc.items():
            pct = 100 * v / len(df)
            print(f"  - {str(k):15s} : {v:10d}  ({pct:6.2f} %)")

        print(f"\n[attack_cat] Nº clases únicas: {ac.nunique(dropna=False)}")
    else:
        print("\n⚠️ No existe la columna attack_cat")

    if has_label:
        lb = df["label"].astype(str).str.strip()
        vc = lb.value_counts(dropna=False)

        print("\n[label] Distribución:")
        for k, v in vc.items():
            pct = 100 * v / len(df)
            print(f"  - {str(k):5s} : {v:10d}  ({pct:6.2f} %)")
    else:
        print("\n⚠️ No existe la columna label")


# ==================================================
# EJECUCIÓN
# ==================================================
for name, path in CSV_FILES.items():
    if Path(path).exists():
        audit_csv(path, name)
    else:
        print("\n" + "="*70)
        print(f"[DATASET] {name}")
        print("❌ Archivo NO encontrado:", path)



[DATASET] UNSW_REAL
[PATH]    /gdrive/MyDrive/Datasets/UNSW_NB15_full_clean.csv
[SHAPE]   (2540047, 45)
[COLUMNAS] attack_cat=True | label=True

[attack_cat] Clases detectadas:
  - Normal          :    2218764  ( 87.35 %)
  - Generic         :     217450  (  8.56 %)
  - Exploits        :      44525  (  1.75 %)
  - Fuzzers         :      24246  (  0.95 %)
  - DoS             :      16353  (  0.64 %)
  - Reconnaissance  :      13987  (  0.55 %)
  - Analysis        :       2677  (  0.11 %)
  - Shellcode       :       1511  (  0.06 %)
  - Backdoors       :        534  (  0.02 %)

[attack_cat] Nº clases únicas: 9

[label] Distribución:
  - 0     :    2218764  ( 87.35 %)
  - 1     :     321283  ( 12.65 %)

[DATASET] UNSW_SYN
[PATH]    /gdrive/MyDrive/Datasets/synthetic_UNSW_NB15_ctgan.csv
[SHAPE]   (440000, 45)
[COLUMNAS] attack_cat=True | label=True

[attack_cat] Clases detectadas:
  - Normal          :     200000  ( 45.45 %)
  - Analysis        :      30000  (  6.82 %)
  - Backdoors      